In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from functions import mean_gen, varx_gen, vary_gen, spherical_data, test_on_integers, learn_parameters

2022-08-09 14:41:16.993442: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-09 14:41:16.993480: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Random Seed
rand_n = 12345

# Single Point on Sphere
n = 250000
angles = np.array([2.5, 1.5])
thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

In [3]:
model_interpolate = tf.keras.models.load_model('3dmodels/discrete_model_mth40_mph40')

for l in model_interpolate.layers:
    l.trainable=False
    
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold = tf.keras.Input(shape=(1,))
simple_linear = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs = tf.keras.Input(shape=(2,))
inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
output = model_interpolate(inputs)

model_paramfinder = Model(inputs = raw_inputs, outputs = output)
model_paramfinder.compile(loss=loss_fn, optimizer='Adam')
model_paramfinder.summary()

2022-08-09 14:41:23.006932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-09 14:41:23.006992: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-09 14:41:23.007016: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-08-09 14:41:23.007305: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like (TFOpLambda)       (None, 2)            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1)            0           tf.ones_like[0][0]               
__________________________________________________________________________________________________
model (Functional)              (None, 2)            2           tf.__operators__.getitem[0][0]   
____________________________________________________________________________________________

In [43]:
test_on_integers(x_train, y_train)

discrete_model_th5_ph5 has loss 0.7342983435112683
discrete_model_th3_ph3 has loss 0.645837751115498
discrete_model_th2_ph2 has loss 0.5913730227368763
discrete_model_th1_ph1 has loss 0.6055138483404225
discrete_model_th0_ph0 has loss 0.6925703341089189


In [44]:
epochs, iterations, batch_size = 1, 50, 300

weights = learn_parameters(model_paramfinder, x_train, y_train, iterations, epochs, batch_size)
angle_errors = (angles - weights) / angles
average_error = np.sum(np.abs(angle_errors)) / 2
print('Average error %:', average_error * 100)

534/534 [==============================] - 2s 3ms/step - loss: 0.5743
0 Fitted result:  tf.Tensor([1.3180417 1.3427593], shape=(2,), dtype=float32)
0.5695312963574065
534/534 [==============================] - 1s 3ms/step - loss: 0.5682
1 Fitted result:  tf.Tensor([1.4388989 1.4383734], shape=(2,), dtype=float32)
0.5681589846236654
534/534 [==============================] - 1s 3ms/step - loss: 0.5677
2 Fitted result:  tf.Tensor([1.4989302 1.4624131], shape=(2,), dtype=float32)
0.5680114224993177
534/534 [==============================] - 1s 3ms/step - loss: 0.5676
3 Fitted result:  tf.Tensor([1.5264585 1.454413 ], shape=(2,), dtype=float32)
0.5679780781467499
534/534 [==============================] - 1s 2ms/step - loss: 0.5678
4 Fitted result:  tf.Tensor([1.5644647 1.4477116], shape=(2,), dtype=float32)
0.5679487338313745
534/534 [==============================] - 1s 3ms/step - loss: 0.5678
5 Fitted result:  tf.Tensor([1.6135077 1.4470404], shape=(2,), dtype=float32)
0.567910963732765

0.5676557347582463
534/534 [==============================] - 2s 3ms/step - loss: 0.5668
49 Fitted result:  tf.Tensor([3.1026282 1.5151695], shape=(2,), dtype=float32)
0.5676520937262617
Average error %: 12.558214366436005


In [4]:
xr = model_interpolate(x_train).numpy().transpose()[0]
y = model_interpolate(x_train)
loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
print(loss)

0.5674771576487369


In [5]:
x_train_mod = np.copy(x_train)
x_train_mod[:, 2] = x_train[:, 2] + .5
x_train_mod[:, 3] = x_train[:, 3]
xr_mod = model_interpolate(x_train_mod).numpy().transpose()[0]
y_mod = model_interpolate(x_train_mod)
loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
print(loss_mod)

0.5676508639838465


In [25]:
x_train

array([[ 1.07633646e-01,  1.70760504e+00,  4.00000000e+00,
         3.00000000e+00],
       [ 1.43298843e+00,  3.84883422e-01,  4.00000000e+00,
         3.00000000e+00],
       [ 3.76967497e-02,  7.15595959e-01,  4.00000000e+00,
         3.00000000e+00],
       ...,
       [ 2.12611480e-03,  6.37581970e-01,  4.00000000e+00,
         3.00000000e+00],
       [ 8.48878054e-01,  5.50897652e-01,  4.00000000e+00,
         3.00000000e+00],
       [-9.53562972e-01,  5.37223246e-02,  4.00000000e+00,
         3.00000000e+00]])